# Load data in Athena

***

## Libraries

In [2]:
import boto3
import sagemaker
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect
import pandas as pd

## Variables

In [3]:
db_name = "sdpd"
Bucket = 'sdpd-bucket' 
region = boto3.Session().region_name
s3_staging_dir = "s3://{}/athena/staging".format(Bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
role = sagemaker.get_execution_role()
s3 = boto3.client('s3') # Create an S3 client
s3_resource = boto3.resource('s3') # Create an S3 resource

print('S3_staging_dir - > ',s3_staging_dir)
print('Conn - > ',conn)
print('Region - > ',region)


S3_staging_dir - >  s3://sdpd-bucket/athena/staging
Conn - >  <pyathena.connection.Connection object at 0x7ff473be0dd0>
Region - >  us-east-1


## Database

### Create

In [4]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(db_name)
pd.read_sql(statement, conn)

statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
print(df_show.head(5))

if db_name in df_show.values:
    ingest_create_athena_db_passed = True
    
%store ingest_create_athena_db_passed

  database_name
0       default
1        dsoaws
2          sdpd
3       watersd
Stored 'ingest_create_athena_db_passed' (bool)


### DB Setup

In [5]:
DB_location = f"""
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' 
LINES TERMINATED BY '\n' 
LOCATION 's3://{Bucket}/SQL'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')
"""

In [6]:
DB_location = f"""
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' 
LINES TERMINATED BY '\n' 
LOCATION 's3://{Bucket}'
"""

In [7]:
def Header(file):
    obj = s3.get_object(Bucket=Bucket, Key=file)
    header = obj['Body'].read(1000).decode().split('\n')[0]
    header_list = header.split(',')
    return header_list

In [8]:
def SQL_Table_Create(db_name,table_name,file):
    create_table_sql = f"CREATE EXTERNAL TABLE IF NOT EXISTS {db_name}.{table_name} ("
    for col in Header(file):
        create_table_sql += f"{col} string,"
    create_table_sql = create_table_sql[:-1] + ")"  # Remove trailing comma and add closing parenthesis
    return create_table_sql

In [9]:
SQL_Table_Create(db_name,'calls','Call_Data/SDPD_Calls_.csv')

'CREATE EXTERNAL TABLE IF NOT EXISTS sdpd.calls (incident_num string,date_time string,day_of_week string,address_number_primary string,address_dir_primary string,address_road_primary string,address_sfx_primary string,address_dir_intersecting string,address_road_intersecting string,address_sfx_intersecting string,call_type string,disposition string,beat string,priority string,Year string)'

In [10]:
SQL_Table_Create(db_name,'Ripa_Stops','Ripa_Stops.csv')

'CREATE EXTERNAL TABLE IF NOT EXISTS sdpd.Ripa_Stops (stop_id string,ori string,agency string,exp_years string,date_stop string,time_stop string,stopduration string,stop_in_response_to_cfs string,officer_assignment_key string,assignment string,intersection string,address_block string,land_mark string,address_street string,highway_exit string,isschool string,school_name string,address_city string,beat string,beat_name string,pid string,isstudent string,perceived_limited_english string,perceived_age string,perceived_gender string,gender_nonconforming string,gend string,gend_nc string,perceived_lgbt string)'

In [11]:
SQL_Calls = """CREATE EXTERNAL TABLE IF NOT EXISTS sdpd.calls (incident_num string,
date_time string,
day_of_week string,
address_number_primary string,
address_dir_primary string,
address_road_primary string,
address_sfx_primary string,
address_dir_intersecting string,
address_road_intersecting string,
address_sfx_intersecting string,
call_type string,
disposition string,
beat string,
priority string,
year int)"""

In [12]:
SQL_Calls

'CREATE EXTERNAL TABLE IF NOT EXISTS sdpd.calls (incident_num string,\ndate_time string,\nday_of_week string,\naddress_number_primary string,\naddress_dir_primary string,\naddress_road_primary string,\naddress_sfx_primary string,\naddress_dir_intersecting string,\naddress_road_intersecting string,\naddress_sfx_intersecting string,\ncall_type string,\ndisposition string,\nbeat string,\npriority string,\nyear int)'

In [13]:
SQL_Stop = """ CREATE EXTERNAL TABLE IF NOT EXISTS sdpd.Ripa_Stops (stop_id string,
ori string,
agency string,
exp_years string,
date_stop string,
time_stop string,
stopduration string,
stop_in_response_to_cfs string,
officer_assignment_key string,
assignment string,
intersection string,
address_block string,
land_mark string,
address_street string,
highway_exit string,
isschool string,
school_name string,
address_city string,
beat string,
beat_name string,
pid string,
isstudent string,
perceived_limited_english string,
perceived_age string,
perceived_gender string,
gender_nonconforming string,
gend string,
gend_nc string,
perceived_lgbt string)"""

In [14]:
SQL_Stop

' CREATE EXTERNAL TABLE IF NOT EXISTS sdpd.Ripa_Stops (stop_id string,\nori string,\nagency string,\nexp_years string,\ndate_stop string,\ntime_stop string,\nstopduration string,\nstop_in_response_to_cfs string,\nofficer_assignment_key string,\nassignment string,\nintersection string,\naddress_block string,\nland_mark string,\naddress_street string,\nhighway_exit string,\nisschool string,\nschool_name string,\naddress_city string,\nbeat string,\nbeat_name string,\npid string,\nisstudent string,\nperceived_limited_english string,\nperceived_age string,\nperceived_gender string,\ngender_nonconforming string,\ngend string,\ngend_nc string,\nperceived_lgbt string)'

In [15]:
SQL_Calls + DB_location

"CREATE EXTERNAL TABLE IF NOT EXISTS sdpd.calls (incident_num string,\ndate_time string,\nday_of_week string,\naddress_number_primary string,\naddress_dir_primary string,\naddress_road_primary string,\naddress_sfx_primary string,\naddress_dir_intersecting string,\naddress_road_intersecting string,\naddress_sfx_intersecting string,\ncall_type string,\ndisposition string,\nbeat string,\npriority string,\nyear int)\nROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' \nLINES TERMINATED BY '\n' \nLOCATION 's3://sdpd-bucket'\n"

### Create Tables

In [16]:
pd.read_sql(SQL_Calls + DB_location, conn)

""


In [17]:
pd.read_sql(SQL_Stop + DB_location, conn)

""


In [18]:
qry = """
CREATE EXTERNAL TABLE IF NOT EXISTS `sdpd`.`Calls_SDPD_2023` (
  `incident_num` string,
  `date_time` string,
  `day_of_week` string,
  `address_number_primary` string,
  `address_dir_primary` string,
  `address_road_primary` string,
  `address_sfx_primary` string,
  `address_dir_intersecting` string,
  `address_road_intersecting` string,
  `address_sfx_intersecting` string,
  `call_type` string,
  `disposition` string,
  `beat` string,
  `priority` string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('field.delim' = ',')
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://sdpd-bucket/2023/'
TBLPROPERTIES ('classification' = 'csv', 'skip.header.line.count'='1');
"""

In [19]:
qry2 = """CREATE EXTERNAL TABLE IF NOT EXISTS `sdpd`.`CallsSDPD` (
  `incident_num` string,
  `date_time` string,
  `day_of_week` string,
  `address_number_primary` string,
  `address_dir_primary` string,
  `address_road_primary` string,
  `address_sfx_primary` string,
  `address_dir_intersecting` string,
  `address_road_intersecting` string,
  `address_sfx_intersecting` string,
  `call_type` string,
  `disposition` string,
  `beat` string,
  `priority` string,
  `year` string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('field.delim' = ',')
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://sdpd-bucket/Call_Data/'
TBLPROPERTIES ('classification' = 'csv','skip.header.line.count'='1');"""

In [20]:
qry = SQL_Table_Create(db_name,'calls','Call_Data/SDPD_Calls_.csv')

In [21]:
addOn = """ ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('field.delim' = ',')
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://sdpd-bucket/Call_Data/''
TBLPROPERTIES ('classification' = 'csv', 'skip.header.line.count'='1');"""

In [22]:
qry = qry + addOn

In [23]:
qry

"CREATE EXTERNAL TABLE IF NOT EXISTS sdpd.calls (incident_num string,date_time string,day_of_week string,address_number_primary string,address_dir_primary string,address_road_primary string,address_sfx_primary string,address_dir_intersecting string,address_road_intersecting string,address_sfx_intersecting string,call_type string,disposition string,beat string,priority string,Year string) ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'\nWITH SERDEPROPERTIES ('field.delim' = ',')\nSTORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'\nLOCATION 's3://sdpd-bucket/Call_Data/''\nTBLPROPERTIES ('classification' = 'csv', 'skip.header.line.count'='1');"

In [24]:
pd.read_sql(qry2, conn)

""


In [25]:
statement = "SHOW TABLES in {}".format(db_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,tab_name
0,calls
1,calls_2023
2,calls_sdpd
3,calls_sdpd_2023
4,callssdpd


In [26]:
statement = """SELECT * FROM {}.{}
    --order by incident_num desc
     LIMIT 100
     """.format(
    db_name, 'CallsSDPD'
)

print(statement)

SELECT * FROM sdpd.CallsSDPD
    --order by incident_num desc
     LIMIT 100
     


In [27]:
df = pd.read_sql(statement, conn)
df.head(5)

,incident_num,date_time,day_of_week,address_number_primary,address_dir_primary,address_road_primary,address_sfx_primary,address_dir_intersecting,address_road_intersecting,address_sfx_intersecting,call_type,disposition,beat,priority,year
0,E18010000001,2018-01-01 00:00:09,0,0,,MONROE,AVE,,PARK,,FD,K,624,2,2018
1,E18010000002,2018-01-01 00:00:41,0,0,,MAIN,ST,,THOR,,AU1,W,511,1,2018
2,E18010000003,2018-01-01 00:00:52,0,4300,,OCEAN,BLV,,,,1016,A,122,2,2018
3,E18010000004,2018-01-01 00:00:55,0,4500,,BOYLSTON,ST,,,,AU1,W,445,1,2018
4,E18010000005,2018-01-01 00:00:56,0,0,,04TH,AVE,,ISLAND,,1151,O,523,3,2018


## Data Quality Report

In [28]:
def Data_Quality_Report(df):

    #Initial table
    freqDF = pd.DataFrame(columns=['Feature',
                                   'Mode',
                                   'Mode Freq.',
                                   'Mode %',
                                   '2nd Mode',
                                   '2nd Mode Freq.',
                                   '2nd Mode %'])
    for col in df.columns:
        freq = df[col].value_counts()
        freqdf = freq.to_frame()
        fRow = freqdf.iloc[0]
        #try:
        secRow = freqdf.iloc[1]
        #except:
        #secRow = 0
        fPrct = fRow[0] / len(df[col])
        #try:
        secPrct = secRow[0] / len(df[col])
        #except:
            #secPrct = 0
        try:
            mode1 = int(fRow.name)
        except:
            mode1 = fRow.name
        try:
            mode2 = int(secRow.name)
        except:
            try:
                mode2 = secRow.name
            except:
                mode2 = 0
        freqDF = freqDF.append({'Feature':col,
                                'Mode':mode1,
                                'Mode Freq.':fRow[0],
                                'Mode %':fPrct,\
                                '2nd Mode':mode2,
                                '2nd Mode Freq.':secRow[0],
                                '2nd Mode %':secPrct},
                                ignore_index=True)

    freqDF = freqDF.set_index('Feature')

    #Nulls, Counts, Cardinality
    NUllFeatures = round(df.isnull().sum() / df.shape[0],4)\
          .sort_values(ascending=False)
    Count = df.count()
    uni = df.nunique()

    #Formating
    NUllFeatures.to_frame(name="% Miss.")
    Count.to_frame(name="Count")
    uni.to_frame()
    result = pd.concat([Count, NUllFeatures,uni], axis=1)
    result.columns =["Count","% Miss.","Card."]
    result = pd.concat([result, freqDF], axis=1)
    result = result.style.format({'% Miss.': "{:.1%}",
                         'Mode %': "{:.0%}",
                         '2nd Mode %': "{:.0%}",
                         'Count': "{:,}",
                         'Card.': "{:,}",
                         'Mode Freq.': "{:,}",
                        '2nd Mode Freq.': "{:,}"})
    return result

In [29]:
DQR = Data_Quality_Report(df)

IndexError: single positional indexer is out-of-bounds

In [ ]:
DQR